# Apache Pulsar - Topic (Log) Compaction
[Apache Pulsar - Concepts and Architecture/Topic Compaction](https://pulsar.apache.org/docs/en/concepts-topic-compaction/)

Pulsar was built with highly scalable persistent storage of message data as a primary objective. Pulsar topics enable you to persistently store as many unacknowledged messages as you need while preserving message ordering. By default, Pulsar stores all unacknowledged/unprocessed messages produced on a topic. Accumulating many unacknowledged messages on a topic is necessary for many Pulsar use cases but it can also be very time intensive for Pulsar consumers to "rewind" through the entire log of messages.\
Pulsar以高度可扩展的消息数据持久存储为主要目标。Pulsar主题使你能够在保持消息排序的同时，持续存储尽可能多的未确认消息。默认情况下，Pulsar存储关于一个主题产生的所有未确认/未处理的消息。对于许多Pulsar用例来说，在一个主题上积累许多未确认的消息是必要的，但是对于Pulsar消费者来说，在整个消息日志中“倒带”也是非常耗费时间的。

For a more practical guide to topic compaction, see the [Topic compaction cookbook](https://pulsar.apache.org/docs/en/cookbooks-compaction).\
有关主题压缩的更多实用指南，请参见[Topic compaction cookbook](https://pulsar.apache.org/docs/en/cookbooks-compaction)。

For some use cases consumers don't need a complete "image" of the topic log. They may only need a few values to construct a more "shallow" image of the log, perhaps even just the most recent value. For these kinds of use cases Pulsar offers __topic compaction__. When you run compaction on a topic, Pulsar goes through a topic's backlog and removes messages that are obscured by later messages, i.e. it goes through the topic on a per-key basis and leaves only the most recent message associated with that key.\
对于某些用例，consumer不需要主题日志的完整“映像”。他们可能只需要几个值来构建日志的更“浅层”映像，甚至可能只需要最近的值。对于这类用例，Pulsar提供了__主题压缩__。当你对一个主题进行压缩时，Pulsar会处理一个主题的积压信息，并删除被后面的信息掩盖的信息，也就是说，它会按照每个键来处理这个主题，只留下与该键相关的最近的信息。

__Pulsar's topic compaction feature__\
Pulsar主题压缩特性
* Allows for faster "rewind" through topic logs\
允许更快地“倒带”主题日志
* Applies only to persistent topics\
仅适用于持久主题
* Triggered automatically when the backlog reaches a certain size or can be triggered manually via the command line. See the [Topic compaction cookbook](https://pulsar.apache.org/docs/en/cookbooks-compaction)\
当积压达到一定大小时自动触发，或者可以通过命令行手动触发。请参阅[Topic compaction cookbook](https://pulsar.apache.org/docs/en/cookbooks-compaction)。
* Is conceptually and operationally distinct from retention and expiry. Topic compaction does, however, respect retention. If retention has removed a message from the message backlog of a topic, the message will also not be readable from the compacted topic ledger.\
在概念上和操作上都不同于保留和期满。然而，主题压缩确实尊重保留。如果retention已从主题的消息积压中删除了消息，则该消息也将无法从压缩主题分类帐中读取。

__Topic compaction example: the stock ticker__\
An example use case for a compacted Pulsar topic would be a stock ticker topic. On a stock ticker topic, each message bears a timestamped dollar value for stocks for purchase (with the message key holding the stock symbol, e.g. AAPL or GOOG). With a stock ticker you may care only about the most recent value(s) of the stock and have no interest in historical data (i.e. you don't need to construct a complete image of the topic's sequence of messages per key). Compaction would be highly beneficial in this case because it would keep consumers from needing to rewind through obscured messages.\
压缩型Pulsar主题的一个例子是股票行情主题。在股票行情主题中，每条信息都有一个带有购买股票时间戳的美元价值（该信息键持有股票代码，如AAPL或GOOG）。使用股票行情，你可能只关心股票的最近价值，而对历史数据没有兴趣（也就是说，你不需要构建主题的每个键的消息序列的完整图像）。压缩在这种情况下是非常有益的，因为它将使使用者不需要倒带模糊的消息。

## How topic compaction works

When topic compaction is triggered via the CLI, Pulsar will iterate over the entire topic from beginning to end. For each key that it encounters the compaction routine will keep a record of the latest occurrence of that key.\
当通过CLI触发主题压缩时，Pulsar将从头到尾迭代整个主题。对于它遇到的每个键，压缩例程将保存该键最近出现的记录。

After that, the broker will create a new BookKeeper ledger and make a second iteration through each message on the topic. For each message, if the key matches the latest occurrence of that key, then the key's data payload, message ID, and metadata will be written to the newly created ledger. If the key doesn't match the latest then the message will be skipped and left alone. If any given message has an empty payload, it will be skipped and considered deleted (akin to the concept of tombstones in key-value databases). At the end of this second iteration through the topic, the newly created BookKeeper ledger is closed and two things are written to the topic's metadata: the ID of the BookKeeper ledger and the message ID of the last compacted message (this is known as the compaction horizon of the topic). Once this metadata is written compaction is complete.\
在此之后，代理将创建一个新的簿记分类帐，并对关于该主题的每个消息进行第二次迭代。对于每个消息，如果键与该键的最近一次匹配，则该键的数据有效负载、消息ID和元数据将写入新创建的分类账。如果该键与最新的不匹配，那么该消息将被跳过而不被处理。如果任何给定的消息有空的有效负载，它将被跳过并认为已删除(类似于键值数据库中的tombstone概念)。年底的第二个迭代通过主题,新创建的簿记员分类帐关闭,两件事都写这个话题的元数据:会计总帐的ID和消息ID的最后压实消息(这被称为压实层的主题)。写入此元数据后，压缩就完成了。

After the initial compaction operation, the Pulsar broker that owns the topic is notified whenever any future changes are made to the compaction horizon and compacted backlog. When such changes occur:\
在初始压缩操作之后，一旦压缩水平和压缩积压发生任何更改，拥有主题的Pulsar broker就会得到通知。当这些变化发生时:
* Clients (consumers and readers) that have read compacted enabled will attempt to read messages from a topic and either:\
启用了读压缩功能的client（消费者和读者）将尝试从主题中读取消息，或者:
 * Read from the topic like normal (if the message ID is greater than or equal to the compaction horizon) or\
 从主题中正常读取(如果消息ID大于或等于压缩范围)或
 * Read beginning at the compaction horizon (if the message ID is lower than the compaction horizon)\
 从压缩水平开始读取(如果消息ID低于压缩水平)